In [0]:
#@title Which environment you are using
#@markdown In order to import the dataset, please put this file in the same directory with the dataset folder <br><br>
#@markdown Which enviroenment you are running for this file?
load_source = "Colab"  #@param ['Colab', 'Jupyter Notebook']
#@markdown --- 

In [0]:
if(load_source=="Colab"):
  from google.colab import drive
  drive.mount("/content/gdrive/")

In [0]:
import zipfile
import os
import re
import numpy as np
#please set the path equal to your directory
base_path="/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3"

In [0]:
#@title Dataset selection
#@markdown Which dataset you want to choose?
selected_dataset = "Kaggle"  #@param ['AUC', 'Kaggle']
#@markdown --- 

In [7]:
%cd {base_path}
if(selected_dataset == "AUC"):
  auc_dir = "AUC_Dataset"
  !mkdir {auc_dir}
  !unzip auc.distracted.driver.dataset_v2.zip -d {auc_dir}
if(selected_dataset == "Kaggle"):
  kaggle_dir="Kaggle_Dataset"
  !mkdir {kaggle_dir}
  if(load_source=="Colab"):
    !unzip state-farm-distracted-driver-detection.zip -d {kaggle_dir}
  else:
    zip_path=os.path.join(base_path,"state-farm-distracted-driver-detection.zip")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(kaggle_dir)

/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3
mkdir: cannot create directory ‘Kaggle_Dataset’: File exists
Archive:  state-farm-distracted-driver-detection.zip
replace Kaggle_Dataset/driver_imgs_list.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Kaggle_Dataset/imgs/test/img_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
import keras
from keras.applications import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import InceptionV3
from keras.callbacks import ModelCheckpoint


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import math
from sklearn import metrics 
import seaborn as sns

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#Configuration
selected_model=InceptionV3
target_size=(150,150)
dataset_dir=os.path.join(base_path,kaggle_dir)
train_dir=os.path.join(dataset_dir,"imgs/train")
test_dir=os.path.join(dataset_dir,"imgs/test")
num_label=10

In [0]:
input_size=list(target_size)
input_size.append(3)
input_size=tuple(input_size)

In [11]:
conv_layers=selected_model(weights="imagenet",include_top=False,input_shape=input_size)

model=keras.models.Sequential()
model.add(conv_layers)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation="relu"))
# model.add()
model.add(keras.layers.Dense(num_label,activation="softmax"))

87916544/87910968 [==============================] - 6s 0us/step


In [12]:
train_datagen=ImageDataGenerator(rescale=1./255)

test_datagen=ImageDataGenerator(rescale=1./255)

# train_datagen=ImageDataGenerator(rescale=1./255,
#                                  )

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=target_size,
                                                  batch_size=60,
                                                  class_mode="categorical"
                                                  )

# test_generator=test_datagen.flow_from_directory(test_dir,
#                                                   target_size=target_size,
#                                                   batch_size=1,
#                                                   class_mode="categorical",
#                                                   shuffle=False
#                                                   )

Found 22424 images belonging to 10 classes.


In [0]:
num_steps=len(train_generator)

In [0]:
ckpt_path=os.path.join(base_path,"exported_model.hdf5")
save_model=ModelCheckpoint(ckpt_path,monitor="train_loss",save_best_only=True,mode="min")

In [0]:
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['acc'])

In [0]:
history=model.fit_generator(train_generator,
                    steps_per_epoch=num_steps,
                    epochs=50,
                    callbacks=[save_model])

Epoch 1/50


In [47]:
train_acc=history.history["acc"]
# val_acc=history.history["val_acc"]
num_epochs=len(train_acc)
plt.figure()
plt.plot(range(num_epochs),train_acc,label="Training Accuracy")
# plt.plot(range(num_epochs),val_acc,label="Validation Accuracy")
plt.title("Train Accuracy")
plt.legend()

train_loss=history.history["loss"]
# val_loss=history.history["val_loss"]
plt.figure()
plt.plot(range(num_epochs),train_loss,label="Training Loss")
# plt.plot(range(num_epochs),val_loss,label="Validation Loss")
plt.title("Train Loss")
plt.legend()

NameError: ignored

In [0]:
y_train=train_generator.classes

In [0]:
distri_train=np.zeros(num_label)
for idx,val in enumerate(y_train):
  distri_train[val]+=1

In [20]:
distri_train

array([2489., 2267., 2317., 2346., 2326., 2312., 2325., 2002., 1911.,
       2129.])

##Predict

In [0]:
import pandas as pd